# Statistical models in R
This notebook is covering:
1. VECM

In [1]:
import pandas as pd
import plotly.express as px
import plotly.graph_objs as go

In [2]:
# rpy2 is a Python package that allows you to run R code from Python
%pip install rpy2

Note: you may need to restart the kernel to use updated packages.


In [3]:
# Load the rpy2 extension to use R in Jupyter
%load_ext rpy2.ipython

The magic function `%%R` is used for running R code in Jupyter

In [4]:
%%R
source("garch.r")
hub1_name <- "ttf"
hub2_name <- "the"
validation_size <- 250
test_size <- 250
window_size <- 5

Loading required package: dplyr

Attaching package: ‘dplyr’

The following objects are masked from ‘package:stats’:

    filter, lag

The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union

Loading required package: zoo

Attaching package: ‘zoo’

The following objects are masked from ‘package:base’:

    as.Date, as.Date.numeric

Loading required package: forecast
Registered S3 method overwritten by 'quantmod':
  method            from
  as.zoo.data.frame zoo 
Loading required package: tseries

    ‘tseries’ version: 0.10-57

    ‘tseries’ is a package for time series analysis and computational
    finance.

    See ‘library(help="tseries")’ for details.

Loading required package: moments
Loading required package: tibble
Loading required package: urca
Loading required package: kableExtra

Attaching package: ‘kableExtra’

The following object is masked from ‘package:dplyr’:

    group_rows

Loading required package: rugarch
Loading required packag

In [5]:
%%R
if (require("foreach") == FALSE) {
  install.packages("foreach")
  library(foreach)
}
if (require("doParallel") == FALSE) {
  install.packages("doParallel")
  library(doParallel)
}

Loading required package: foreach
Loading required package: doParallel
Loading required package: iterators


In [ ]:
%%R
source("garch.r")
validation_size <- 250
test_size <- 250
# Set up the parallel backend to use multiple cores
num_cores <- 12 
cl <- makeCluster(num_cores)
registerDoParallel(cl)
clusterEvalQ(cl, library(tsDyn))
clusterEvalQ(cl, library(rugarch))

clusterEvalQ(cl, source("garch.r"))

clusterExport(cl, varlist=c("garch_system", "garch_validation_predictions", "garch_test_predictions", "validation_size", "test_size"))

pairs <- list(c("ttf", "the"), 
        c("ttf", "nbp"), 
        c("the", "nbp")
)
window_sizes <- c(5)
models <- c("fiGARCH")
distributions <- c("sstd")

results <- foreach(i = 1:length(pairs), .combine = c, .multicombine = TRUE) %:%
            foreach(j = 1:length(models), .combine = c) %:%
            foreach(d = 1:length(distributions), .combine = c) %:%
           foreach(k = 1:length(window_sizes), .combine = c) %dopar% {

               pair <- pairs[[i]]
               hub1_name <- pair[1]
               hub2_name <- pair[2]
               window_size <- window_sizes[k]
                model <- models[j]
                dist <- distributions[d]
               print(paste("Starting", hub1_name, hub2_name,model, dist, window_size))
               run <- garch_system(hub1_name, hub2_name, model, dist, validation_size, test_size, window_size, garch_order = c(2,1), verbose=FALSE, save=TRUE)
               paste("Task", i,j, "done")
           }
print(results)
stopCluster(cl)

[1] "Task 1 1 done" "Task 2 1 done" "Task 3 1 done"


In [22]:
%%R
pairs <- list(c("ttf", "the"), 
            c("ttf", "nbp"), 
            c("the", "nbp")
            )
window_sizes <- c(5, 10)
for (i in 1:length(pairs)) {
    for(k in 1:length(window_sizes)) {
        pair <- pairs[[i]]
        hub1_name <- pair[1]
        hub2_name <- pair[2]
        window_size <- window_sizes[k]
        garch_system(hub1_name, hub2_name, "sGARCH", "norm", validation_size, test_size, window_size, verbose=FALSE, save=TRUE)
        print(paste("Finished", hub1_name, hub2_name, window_size))
    }
}

[1] "Finished ttf the 5"
[1] "Finished ttf the 10"
[1] "Finished ttf nbp 5"
[1] "Finished ttf nbp 10"
[1] "Finished the nbp 5"
[1] "Finished the nbp 10"


In addition: Warning messages:
1: In arima(data, order = c(modelinc[2], 0, modelinc[3]), include.mean = modelinc[1],  :
  possible convergence problem: optim gave code = 1
2: In arima(data, order = c(modelinc[2], 0, modelinc[3]), include.mean = modelinc[1],  :
  possible convergence problem: optim gave code = 1
3: In arima(data, order = c(modelinc[2], 0, modelinc[3]), include.mean = modelinc[1],  :
  possible convergence problem: optim gave code = 1
4: In arima(data, order = c(modelinc[2], 0, modelinc[3]), include.mean = modelinc[1],  :
  possible convergence problem: optim gave code = 1
5: In arima(data, order = c(modelinc[2], 0, modelinc[3]), include.mean = modelinc[1],  :
  possible convergence problem: optim gave code = 1
6: In arima(data, order = c(modelinc[2], 0, modelinc[3]), include.mean = modelinc[1],  :
  possible convergence problem: optim gave code = 1
7: In arima(data, order = c(modelinc[2], 0, modelinc[3]), include.mean = modelinc[1],  :
  possible convergence problem: op

In [5]:
%%R
garch_output <- garch_system("ttf", "the", 1, 1, 5, verbose=TRUE, save=TRUE)

In [ ]:
%%R
source("garch.r")
hub1_name <- "ttf"
hub2_name <- "the"

hub_prices <- list(
    nbp = read.csv("../../data/interpolated/nbp_close_interpolated.csv"),
    peg = read.csv("../../data/interpolated/peg_close_interpolated.csv"),
    the = read.csv("../../data/interpolated/the_close_interpolated.csv"),
    ttf = read.csv("../../data/interpolated/ttf_close_interpolated.csv"),
    ztp = read.csv("../../data/interpolated/ztp_close_interpolated.csv")
)

hub1 <- hub_prices[[hub1_name]]
hub2 <- hub_prices[[hub2_name]]

hubs <- data.frame(hub1 = hub1$CLOSE, hub2 = hub2$CLOSE)

#garch_validation_predictions <- garch_validation_predictions(hubs, window_size = window_size, validation_size = validation_size, test_size = test_size, lags = lags)
garch_predictions <- garch_test_predictions(hubs, window_size = window_size, test_size = test_size, lags = lags)

[1] 1


[1] 2 2 0 0 1 0 0
[1] 2
[1] 2 2 0 0 1 0 0
[1] 3
[1] 2 2 0 0 1 0 0
[1] 4
[1] 2 2 0 0 1 0 0
[1] 5
[1] 2 2 0 0 1 0 0
[1] 6
[1] 2 2 0 0 1 0 0
[1] 7
[1] 2 2 0 0 1 0 0
[1] 8
[1] 2 2 0 0 1 0 0
[1] 9
[1] 2 2 0 0 1 0 0
[1] 10
[1] 2 2 0 0 1 0 0
[1] 11
[1] 2 2 0 0 1 0 0
[1] 12
[1] 2 2 0 0 1 0 0
[1] 13
[1] 2 2 0 0 1 0 0
[1] 14
[1] 2 2 0 0 1 0 0
[1] 15
[1] 2 2 0 0 1 0 0
[1] 16
[1] 2 2 0 0 1 0 0
[1] 17
[1] 2 2 0 0 1 0 0
[1] 18
[1] 2 2 0 0 1 0 0
[1] 19
[1] 2 2 0 0 1 0 0
[1] 20
[1] 2 2 0 0 1 0 0
[1] 21
[1] 2 2 0 0 1 0 0
[1] 22
[1] 2 2 0 0 1 0 0
[1] 23
[1] 2 2 0 0 1 0 0
[1] 24
[1] 2 2 0 0 1 0 0
[1] 25
[1] 2 2 0 0 1 0 0
[1] 26
[1] 2 2 0 0 1 0 0
[1] 27
[1] 2 2 0 0 1 0 0
[1] 28
[1] 2 2 0 0 1 0 0
[1] 29
[1] 2 2 0 0 1 0 0
[1] 30
[1] 2 2 0 0 1 0 0
[1] 31
[1] 2 2 0 0 1 0 0
[1] 32
[1] 2 2 0 0 1 0 0
[1] 33
[1] 2 2 0 0 1 0 0
[1] 34
[1] 2 2 0 0 1 0 0
[1] 35
[1] 2 2 0 0 1 0 0
[1] 36
[1] 2 2 0 0 1 0 0
[1] 37
[1] 2 2 0 0 1 0 0
[1] 38
[1] 2 2 0 0 1 0 0
[1] 39
[1] 2 2 0 0 1 0 0
[1] 40
[1] 2 2 0 0 1 0 0
[1] 41
[1] 2 2 

RInterpreterError: Failed to parse and evaluate line 'source("garch.r")\nhub1_name <- "ttf"\nhub2_name <- "the"\n\nhub_prices <- list(\n    nbp = read.csv("../../data/interpolated/nbp_close_interpolated.csv"),\n    peg = read.csv("../../data/interpolated/peg_close_interpolated.csv"),\n    the = read.csv("../../data/interpolated/the_close_interpolated.csv"),\n    ttf = read.csv("../../data/interpolated/ttf_close_interpolated.csv"),\n    ztp = read.csv("../../data/interpolated/ztp_close_interpolated.csv")\n)\n\nhub1 <- hub_prices[[hub1_name]]\nhub2 <- hub_prices[[hub2_name]]\n\nhubs <- data.frame(hub1 = hub1$CLOSE, hub2 = hub2$CLOSE)\n\n# Perform the GARCH model\n#garch_validation_predictions <- garch_validation_predictions(hubs, window_size = window_size, validation_size = validation_size, test_size = test_size, lags = lags)\ngarch_predictions <- garch_test_predictions(hubs, window_size = window_size, test_size = test_size, lags = lags)\n'.
R error message: 'Error: Exiting'
R stdout:
Er